# Weather API Observations
1. Temperature decreases with increase in latitude in the northern hemisphere
2. Temperature increases as you move away from the equator in the Northern hemisphere
3. Temperature increases as you get closer to the equator in the Southern hemisphere

In [ ]:
#Import all necessary Libraries
import pandas as pd
import requests
import json
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from citipy import citipy
import numpy as np
import pandas as pd
# Weather API Key
from config import weather_api
import scipy.stats as st

# Output File (CSV)
output_data_file = "cities.csv"

In [ ]:
#Randomly select at least 500 unique (non-repeat) cities based on latitude and longitude.
#WeatherPy
# In this example, you'll be creating a Python script to visualize the weather of 500+ cities across the world of varying distance from the equator. To accomplish this, you'll be utilizing a simple Python library, the OpenWeatherMap API,
# and a little common sense to create a representative model of weather across world cities.

#Step 1: Get range of longitude and latitude.
#Latitudes range from -90 to 90, and longitudes range from -180 to 180
#generate random list for latitudes and longitudes 

latitude = []
longitude = []

for each_lat in np.random.randint(-90,90,1500):
    latitude.append(each_lat)

for each_lon in np.random.randint(-180,180,1500):
    longitude.append(each_lon)

    
#Create and variable called latitude_and_Longitude and store all random latitudes and longitudes generated

latitude_and_Longitude = tuple(zip(latitude,longitude))

In [ ]:

cities = []
country = []

for lat,lon in latitude_and_Longitude:
    city = citipy.nearest_city(lat,lon)
  

    # Replace spaces with %20 to create url correctly 
    #city = city.replace(" ", "%20")
    cityName = city.city_name
    # If the city is unique, then add it to a our cities list
    if city not in cities:

        cities.append(cityName)
        
        country.append(city.country_code)

# Print the city count to confirm sufficient count
len(cities)
    
print('The script generated {} cities using random numbers.'.format(len(set(cities))))

In [ ]:
#create a dataframe to store all the cities and countries generated with citipy

df_countries = pd.DataFrame({'Cities': cities,
              'Country': country})

#add additional blank columns to store information from openweathermap api

df_countries['Latitude'] = ''
df_countries['Longitude'] = ''
df_countries['Temperature (F)'] = ''
df_countries['Humidity (%)'] = ''
df_countries['Cloudiness (%)'] = ''
df_countries['Wind Speed (mph)'] = ''

In [ ]:
#drop all duplicates in column Cities.

df_countries = df_countries.drop_duplicates(subset=['Cities'],keep='first')

In [ ]:
#loop through all rows and fill in values for blank columns in dataframe

for index,row in df_countries.iterrows():
    city_name = row['Cities']
    Country_id = row['Country']
    
    
    #api call for all Cities 
    api_url = "http://api.openweathermap.org/data/2.5/forecast" \
    "?q={},{}&units=IMPERIAL&mode=json&APPID={}".format(city_name,Country_id,weather_api)
    
    
    country_info = requests.get(api_url).json()
    
    #print all api url's for each city
    print(api_url)
    
    
    try:
        df_countries.set_value(index,'Latitude',country_info['city']['coord']['lat'])
        df_countries.set_value(index,'Longitude',country_info['city']['coord']['lon'])
        df_countries.set_value(index,'Temperature (F)',country_info['list'][0]['main']['temp'])
        df_countries.set_value(index,'Humidity (%)',country_info['list'][0]['main']['humidity'])
        df_countries.set_value(index,'Cloudiness (%)',country_info['list'][0]['clouds']['all'])
        df_countries.set_value(index,'Wind Speed (mph)',country_info['list'][0]['wind']['speed'])
        
    except KeyError:
        df_countries.set_value(index,'Latitude',np.nan)
        df_countries.set_value(index,'Longitude',np.nan)
        df_countries.set_value(index,'Temperature (F)',np.nan)
        df_countries.set_value(index,'Humidity (%)',np.nan)
        df_countries.set_value(index,'Cloudiness (%)',np.nan)
        df_countries.set_value(index,'Wind Speed (mph)',np.nan)
        
        print('Missing weather information...skip')

In [ ]:
#change all data recieved from openweathermap api to numerical data
df_countries['Latitude'] = pd.to_numeric(df_countries['Latitude'])
df_countries['Longitude'] = pd.to_numeric(df_countries['Longitude'])
df_countries['Temperature (F)'] = pd.to_numeric(df_countries['Temperature (F)'])
df_countries['Humidity (%)'] = pd.to_numeric(df_countries['Humidity (%)'])
df_countries['Cloudiness (%)'] = pd.to_numeric(df_countries['Cloudiness (%)'])
df_countries['Wind Speed (mph)'] = pd.to_numeric(df_countries['Wind Speed (mph)'])

In [ ]:
#display dataframe df_countries after openweathermap api calls 
df_countries.head()

In [ ]:
#display dataframe df_countries after openweathermap api calls 
#df_countries.head(100)
print(df_countries)

In [ ]:
#Convert Raw Data to DataFrame
#Export the city data into a .csv.
df_countries.to_csv(output_data_file,index=False)  
#Display the DataFrame
df_countries.head()

In [ ]:
#Latitude vs. Temperature Plot
plt.scatter(df_countries['Latitude'], df_countries['Temperature (F)'],color = 'y')

plt.xlabel('Latitude')
plt.ylabel('Temperature (F)')
plt.title('Latitude vs. Temperature(f) (2020-01-24)')
plt.savefig("CityLatitudeVsTemperature.png")


In [ ]:
#Latitude vs. Humidity Plot
plt.scatter(df_countries['Latitude'], df_countries['Humidity (%)'])

plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.title('Latitude vs. Humidity (%) (2020-01-24)')
plt.savefig("CityLatitudeVsHumidity.png")



In [ ]:
#Latitude vs. Cloudiness Plot

plt.scatter(df_countries['Latitude'], df_countries['Cloudiness (%)'])

plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
plt.title('Latitude vs. Cloudiness (%) (2020-01-24)')
plt.savefig("CityLatitudeVsCloudiness.png")



In [ ]:
plt.scatter(df_countries['Latitude'], df_countries['Wind Speed (mph)'])

plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.title('Latitude vs. Wind Speed (mph)(2020-01-24)')
plt.savefig("CityLatitudeVsWindSpeed.png")



In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

def linear_agression(x,y):
    print(f"The r-squared is : {round(st.pearsonr(x, y)[0],2)}")
    (slope, intercept, rvalue, pvalue, stderr) = st.linregress(x, y)
    regress_values = x * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    plt.scatter(x, y)
    plt.plot(x,regress_values,"r-")
    return line_eq

# Define a fuction for annotating
def annotate(line_eq, a, b):
    plt.annotate(line_eq,(a,b),fontsize=15,color="red")

In [ ]:
#Northern Hemisphere - Temp vs. Latitude Linear Regression
#Your next objective is to run linear regression on each relationship, 
#only this time separating them into Northern Hemisphere 
#greater than or equal to 0 degrees latitude) and Southern Hemisphere (less than 0 degrees latitude):

df_countries_North = df_countries[df_countries["Latitude"] >= 0]
df_countries_South = df_countries[df_countries["Latitude"] < 0]

In [ ]:
# Call an function #1
equation = linear_agression(df_countries_North["Latitude"], df_countries_North["Temperature (F)"])

# Call an function #2
annotate(equation, 0, 0)

# Set a title
plt.title("Northern Hemisphere - Max Temp vs. Latitude Linear Regression")

# Set xlabel
plt.xlabel("Latitude")

# Set ylabel
plt.ylabel("Temp (F)")

# Save the figure
plt.savefig("Northern Hemisphere - Temp vs. Latitude Linear Regression.png")

plt.xticks(size=12)
plt.yticks(size=12)



In [ ]:
#Southern Hemisphere - Max Temp vs. Latitude Linear Regression
# Call an function #1
equation = linear_agression(df_countries_South["Latitude"],df_countries_South["Temperature (F)"])

# Call an function #2
annotate(equation, -30, 10)

# Set a title
plt.title("Southern Hemisphere - Max Temp vs. Latitude Linear Regression")

# Set xlabel
plt.xlabel("Latitude")

# Set ylabel
plt.ylabel("Max Temp (F)")

# Save the figure
plt.savefig("Southern Hemisphere - Max Temp vs. Latitude Linear Regression.png")

plt.xticks(size=12)
plt.yticks(size=12)



In [ ]:
# Call an function #1
equation = linear_agression(df_countries_North["Latitude"], df_countries_North["Humidity (%)"])

# Call an function #2
annotate(equation, 40, 15)

# Set a title
plt.title("Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression")

# Set xlabel
plt.xlabel("Latitude")

# Set ylabel
plt.ylabel("Humidity (%)")

# Save the figure
plt.savefig("Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression.png")



In [ ]:
#Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

# Call an function #1
equation = linear_agression(df_countries_South["Latitude"], df_countries_South["Humidity (%)"])

# Call an function #2
annotate(equation, -40, 50)

# Set title
plt.title("Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression")

# Set xlabel
plt.xlabel("Latitude")

# Set ylabel
plt.ylabel("Humidity (%)")

# Save the figure
plt.savefig("Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression.png")

In [ ]:

equation = linear_agression(df_countries_North["Latitude"], df_countries_North["Cloudiness (%)"])

# Call an function #2
annotate(equation, 30, 40)

# Set a title
plt.title("Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression")

# Set xlabel
plt.xlabel("Latitude")

# Set ylabel
plt.ylabel("Cloudiness (%)")

# Save the figure
plt.savefig("Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression.png")

In [ ]:
#Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression


# Call an function #1
equation = linear_agression(df_countries_South["Latitude"], df_countries_South["Cloudiness (%)"])

# Call an function #2
annotate(equation, -30, 40)

# Set a title
plt.title("Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression")

# Set xlabel
plt.xlabel("Latitude")

# Set ylabel
plt.ylabel("Cloudiness (%)")

# Save the figure
plt.savefig("Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression.png")

In [ ]:
# Call an function #1
equation = linear_agression(df_countries_North["Latitude"], df_countries_North["Wind Speed (mph)"])

# Call an function #2
annotate(equation, 40, 20)

# Set  title
plt.title("Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression")

# Set xlabel
plt.xlabel("Latitude")

# Set ylabel
plt.ylabel("Wind Speed (mph)")

# Save the figure
plt.savefig("Northern Hemisphere - Wind Speed vs. Latitude Linear Regression.png")

In [ ]:
# Call an function #1
equation = linear_agression(df_countries_South["Latitude"], df_countries_South["Wind Speed (mph)"])

# Call an function #2
annotate(equation, -30, 5)

# Set  title
plt.title("Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression")

# Set xlabel
plt.xlabel("Latitude")

# Set ylabel
plt.ylabel("Wind Speed (mph)")

# Save the figure
plt.savefig("Southern Hemisphere - Wind Speed vs. Latitude Linear Regression.png")